# SQL for Data Science

Data is rarely handed to you in a perfect CSV file. In the real world, it lives in **Relational Databases**. To get this data, you need to speak the database's language: **SQL** (Structured Query Language).

Think of a database as a collection of Excel sheets (called "tables") that are linked together. SQL lets you ask questions like "Give me all customers who live in Vienna" or "Calculate the average sales for last month."

## Learning Objectives
- **Understand SQL Basics**: Learn the core commands `SELECT` (to pick columns) and `WHERE` (to filter rows).
- **Connect with Python**: Use the `sqlite3` library to talk to a database from your Python code.
- **Load into Pandas**: The ultimate goal—getting the data out of the database and into a Pandas DataFrame so you can analyze it.

In [1]:
import sqlite3
import pandas as pd

# Connect to an in-memory database
conn = sqlite3.connect(":memory:")
cursor = conn.cursor()

# Create a table
cursor.execute("""
CREATE TABLE employees (
    id INTEGER PRIMARY KEY,
    name TEXT,
    department TEXT,
    salary INTEGER
)
""")

# Insert some data
data = [
    (1, "Alice", "Legal", 60000),
    (2, "Bob", "IT", 70000),
    (3, "Charlie", "Legal", 80000),
    (4, "David", "HR", 55000)
]
cursor.executemany("INSERT INTO employees VALUES (?, ?, ?, ?)", data)
conn.commit()
print("Database created and populated.")

Database created and populated.


## 2. Querying with SQL

Now that we have data, let's ask questions. The most important command is `SELECT`.

*   **`SELECT *`**: The asterisk `*` means "all columns".
*   **`FROM employees`**: Tells the database which table to look in.
*   **`WHERE department = 'Legal'`**: This is a **filter**. It only gives us rows where the condition is true.

**Analogy:** Imagine ordering food. "I want everything (`SELECT *`) from the menu (`FROM menu`) where the type is vegetarian (`WHERE type = 'vegetarian'`)."

In [ ]:
# Run a query
cursor.execute("SELECT * FROM employees WHERE department = 'Legal'")
rows = cursor.fetchall()
for row in rows:
    print(row)
# Query executed

## 3. SQL to Pandas

In the previous step, we got a list of raw rows. This is okay, but Data Scientists prefer working with **DataFrames**.

Pandas has a magic function called `read_sql`. You give it your SQL query and the database connection, and it returns a perfectly formatted DataFrame. This is how you will work 99% of the time.

In [ ]:
# Read SQL directly into a DataFrame
df = pd.read_sql("SELECT * FROM employees", conn)
df

## 4. Aggregation in SQL

Sometimes the data is too big to load into Python (e.g., 100 million rows). In these cases, you should ask the database to summarize the data *before* sending it to you.

*   **`GROUP BY`**: Groups rows together (e.g., "Put all 'IT' employees in one pile and all 'Legal' employees in another").
*   **`AVG(salary)`**: Calculates the average for each pile.

This is much faster than loading all 100 million rows into Python and calculating the average there.

In [ ]:
query = """
SELECT department, AVG(salary) as avg_salary
FROM employees
GROUP BY department
"""
df_agg = pd.read_sql(query, conn)
df_agg